In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import preprocessing
from sklearn import linear_model
import statsmodels.formula.api as smf
%matplotlib inline

In [46]:
law_enf = pd.read_excel("okle_new_york.xls", skiprows=4,
                       names=["City", "Population", "ViolentCrime",
                             "Murder", "RapeR", "RapeL", "Robbery",
                             "AggravatedAssault", "PropertyCrime",
                             "Burglary", "Larceny-theft", "MotorTheft",
                             "Arson"])
law_enf_2014 = pd.read_excel("okle_new_york_2014.xls", skiprows=4,
                            names=["City", "Population", "ViolentCrime",
                             "Murder", "RapeR", "RapeL", "Robbery",
                             "AggravatedAssault", "PropertyCrime",
                             "Burglary", "Larceny-theft", "MotorTheft",
                             "Arson"])

law_enf = law_enf.rename(columns=lambda x:x.strip())
law_enf = law_enf.rename(columns=lambda x: x.replace('\n', ''))
law_enf = law_enf.rename(columns=lambda x: x.replace(' ', ''))

law_enf_2014 = law_enf_2014.rename(columns=lambda x:x.strip())
law_enf_2014 = law_enf_2014.rename(columns=lambda x: x.replace('\n', ''))
law_enf_2014 = law_enf_2014.rename(columns=lambda x: x.replace(' ', ''))

In [45]:
law_enf.head()

,City,Population,ViolentCrime,Murder,RapeR,RapeL,Robbery,AggravatedAssault,PropertyCrime,Burglary,Larceny-theft,MotorTheft,Arson
0,Adams Village,1861.0,0.0,0.0,NaN,0.0,0.0,0.0,12.0,2.0,10.0,0.0,0.0
1,Addison Town and Village,2577.0,3.0,0.0,NaN,0.0,0.0,3.0,24.0,3.0,20.0,1.0,0.0
2,Akron Village,2846.0,3.0,0.0,NaN,0.0,0.0,3.0,16.0,1.0,15.0,0.0,0.0
3,Albany,97956.0,791.0,8.0,NaN,30.0,227.0,526.0,4090.0,705.0,3243.0,142.0,NaN
4,Albion Village,6388.0,23.0,0.0,NaN,3.0,4.0,16.0,223.0,53.0,165.0,5.0,NaN


In [28]:
law_enf_2014.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1851.0,0.0,0.0,NaN,0.0,0.0,0.0,11.0,1.0,10.0,0.0,0.0
1,Addison Town and Village,2568.0,2.0,0.0,NaN,0.0,1.0,1.0,49.0,1.0,47.0,1.0,0.0
2,Afton Village4,820.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,Akron Village,2842.0,1.0,0.0,NaN,0.0,0.0,1.0,17.0,0.0,17.0,0.0,0.0
4,Albany4,98595.0,802.0,8.0,54.0,NaN,237.0,503.0,3888.0,683.0,3083.0,122.0,12.0


In [29]:
law_enf['Population^2'] = law_enf['Population'] ** 2
law_enf['Murder'] = law_enf['Murder and nonnegligent manslaughter'] > 0
law_enf['Robbery'] = law_enf['Robbery'] > 0

law_enf_2014['Population^2'] = law_enf_2014['Population'] ** 2
law_enf_2014['Murder'] = law_enf_2014['Murder and nonnegligent manslaughter'] > 0
law_enf_2014['Robbery'] = law_enf_2014['Robbery'] > 0

In [30]:
law_enf['Murder'] = law_enf['Murder and nonnegligent manslaughter'] > 0

In [31]:
law_enf['Robbery'] = law_enf['Robbery'] > 0

In [32]:
law_enf[['Population', 'Population^2', 'Murder', 'Robbery']].head()

,Population,Population^2,Murder,Robbery
0,1861.0,3.463321e+06,False,False
1,2577.0,6.640929e+06,False,False
2,2846.0,8.099716e+06,False,False
3,97956.0,9.595378e+09,True,True
4,6388.0,4.080654e+07,False,True


In [33]:
columns = ['Population', 'Population^2', 'Murder', 'Robbery']
columns_2014 = ['Population', 'Population^2', 'Murder', 'Robbery']

In [34]:
law_enf.loc[law_enf[columns].isnull().any(axis=1),]

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3,Population^2,Murder
348,1 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
349,2 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
350,3 The FBI does not publish arson data unless i...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [35]:
law_enf_2014.loc[law_enf_2014[columns].isnull().any(axis=1),]

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3,Population^2,Murder
369,1 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
370,2 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
371,3 The FBI does not publish arson data unless i...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
372,4 This agency began the year submitting rape d...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
373,5 The FBI determined that the agency's data we...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
374,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
375,,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [36]:
law_enf = law_enf.iloc[:348]
law_enf_2014 = law_enf_2014.iloc[:369]

In [37]:
print(law_enf[law_enf['Property crime'] > 100000])
print(law_enf_2014[law_enf_2014['Property crime'] > 100000])

         City  Population  Violent crime  \
216  New York   8396126.0        52384.0   

     Murder and nonnegligent manslaughter  Rape (revised definition)1  \
216                                 335.0                         NaN   

     Rape (legacy definition)2  Robbery  Aggravated assault  Property crime  \
216                     1112.0     True             31767.0        141971.0   

     Burglary  Larceny- theft  Motor vehicle theft  Arson3  Population^2  \
216   16606.0        117931.0               7434.0     NaN  7.049493e+13   

     Murder  
216    True  
          City  Population  Violent crime  \
227  New York4   8473938.0        50564.0   

     Murder and nonnegligent manslaughter  Rape (revised definition)1  \
227                                 333.0                      2190.0   

     Rape (legacy definition)2  Robbery  Aggravated assault  Property crime  \
227                        NaN     True             31460.0        135747.0   

     Burglary  Larceny- the

In [38]:
law_enf_wo_ny = law_enf.drop(216)
law_enf_2014_wo_ny = law_enf_2014.drop(227)

In [41]:
linear_formula = "Property crime ~ Population"
lm = smf.ols(formula=linear_formula, data=law_enf_wo_ny).fit()

SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
def mixed_selection():
    linear_formula = "Property crime ~ Population"

In [17]:
regr = linear_model.LinearRegression()
Y = law_enf['Property crime'].values.reshape(-1, 1)
X = law_enf[['Population','Population^2','Murder','Robbery']]
regr.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[  3.46570268e-02  -2.11108019e-09   1.51866535e+01  -9.62774363e+01]]

Intercept: 
 [-109.57533562]

R-squared:
0.996124710499


Coefficients: 
 [[1.29510871e-02 1.14920528e-07 1.95038666e+02 9.28205845e+01]]

Intercept: 
 [-16.81586632]

R-squared:
0.8444937611173492

In [19]:
law_enf.corr()

,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3,Population^2,Murder
Population,1.000000,0.998854,0.986758,NaN,0.990839,0.064371,0.999134,0.996265,0.970578,0.997946,0.992421,0.701761,0.998264,0.162309
Violent crime,0.998854,1.000000,0.992098,NaN,0.994985,0.057714,0.999942,0.998059,0.978150,0.998762,0.996271,0.917793,0.996575,0.165904
Murder and nonnegligent manslaughter,0.986758,0.992098,1.000000,NaN,0.997746,0.069557,0.990843,0.995403,0.993733,0.993175,0.997992,0.963200,0.979574,0.211661
Rape (revised definition)1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rape (legacy definition)2,0.990839,0.994985,0.997746,NaN,1.000000,0.076546,0.994140,0.997843,0.992442,0.996260,0.998764,0.926301,0.984331,0.201359
Robbery,0.064371,0.057714,0.069557,NaN,0.076546,1.000000,0.056878,0.079318,0.098352,0.076913,0.070604,0.165537,0.043983,0.313271
Aggravated assault,0.999134,0.999942,0.990843,NaN,0.994140,0.056878,1.000000,0.997622,0.976288,0.998561,0.995403,0.888278,0.997248,0.163747
Property crime,0.996265,0.998059,0.995403,NaN,0.997843,0.079318,0.997622,1.000000,0.987137,0.999666,0.998302,0.920765,0.991027,0.190670
Burglary,0.970578,0.978150,0.993733,NaN,0.992442,0.098352,0.976288,0.987137,1.000000,0.982700,0.991464,0.952121,0.958563,0.240979
Larceny- theft,0.997946,0.998762,0.993175,NaN,0.996260,0.076913,0.998561,0.999666,0.982700,1.000000,0.996878,0.893610,0.993778,0.182823


In [20]:
regr = linear_model.LinearRegression()
Y = law_enf['Property crime'].values.reshape(-1, 1)
X = law_enf[['Population^2','Murder','Robbery']]
regr.fit(X, Y)

print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[  1.99256120e-09   1.18406016e+03   3.00052598e+02]]

Intercept: 
 [ 40.08610471]

R-squared:
0.985987120403


In [23]:
law_enf_wo_ny = law_enf.drop(216)

In [25]:
regr = linear_model.LinearRegression()
Y = law_enf_wo_ny['Property crime'].values.reshape(-1, 1)
X = law_enf_wo_ny[['Population', 'Population^2','Murder','Robbery']]
regr.fit(X, Y)

print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))

# TODO try sklearn RANSAC on data with outlier


Coefficients: 
 [[  1.29510871e-02   1.14920528e-07   1.95038666e+02   9.28205845e+01]]

Intercept: 
 [-16.81586632]

R-squared:
0.844493761117


In [26]:
from sklearn.linear_model import RANSACRegressor